# 导入工具

In [56]:
import warnings
warnings.filterwarnings('ignore')
import gc
from numba import jit
from tqdm import tqdm_notebook, tqdm
tqdm.pandas()
import sys
import os
import json
import logging
logging.basicConfig(format="%(asctime)s: %(levelname)s: %(message)s", level=logging.INFO)

import numpy as np
np.set_printoptions(suppress=True, precision=3)
import pandas as pd
pd.set_option('display.max_columns', None)
import pandas_profiling
import seaborn as sns
sns.set()
import matplotlib.pyplot as  plt
import matplotlib.patches as patches
plt.rcParams['font.sans-serif'] = ['SimHei']
%matplotlib inline

from sklearn.decomposition import FactorAnalysis, FastICA, PCA, TruncatedSVD
from sklearn.manifold import Isomap, TSNE
import umap

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier, Pool, cv

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier, VotingClassifier
from sklearn.preprocessing import Imputer, Normalizer, scale, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.feature_selection import RFECV, SelectKBest, chi2
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from scipy.signal import hilbert, hann, convolve
from scipy import stats
import scipy.spatial.distance as dist

from collections import Counter
from statistics import mode
from itertools import product
import ast
from datetime import timedelta,datetime
import time
import datetime

# 数据初检
- 正式处理所有数据时，可将所有训练数据去掉label后拼接，再与测试数据拼接一同处理

In [3]:
train_df1 = pd.read_table('data/round1/train.txt')  # 1,000,000 samples
train_df2 = pd.read_table('data/round2/train.txt')  # 5,000,000 samples

In [18]:
test_df1 = pd.read_table('data/round1/test.txt')  # 100,000 samples
test_df2a = pd.read_table('data/round2/round2_iflyad_anticheat_testdata_feature_A.txt')  # 1,000,000 samples in A
test_df2b = pd.read_table('data/round2/round2_iflyad_anticheat_testdata_feature_B.txt')  # 1,000,000 samples in B

In [6]:
train_all = pd.concat([train_df1, train_df2], ignore_index=True)
test_all = pd.concat([test_df1, test_df2a, test_df2b], ignore_index=True)
full_train_no_label = train_all.drop(['label'], axis=1)
# 正式处理时对所有数据的features同时处理，之后根据各自的size分出不同的数据集
full_train_test = pd.concat([full_train_no_label, test_all], ignore_index=True)  # 7,100,000 samples without label
del train_df1
del train_df2

- 只跑demo的话可以只用一小部分数据

In [6]:
train_sample = 10000
test_sample = 1000
df_reader = pd.read_table('data/round1/train.txt', iterator=True)
full_data = df_reader.get_chunk(train_sample+test_sample)

train_sample = full_data[:train_sample]  # 0:10000
test_sample = full_data[-test_sample:]  # 10000:11000

# 待数据处理完之后分离特征与标签，否则对full_data的处理不会影响到train_feature与test_feature
# train_feature = train_sample.drop(['label'], axis=1)
# train_label = train_sample.label

# test_feature = test_sample.drop(['label'], axis=1)
# test_label = test_sample.label

## 统一`make`标签

In [ ]:
# 统一make标签
data = full_data
data.loc[data['make'].str.contains('长虹', na=False), 'make'] = 'changhong'
data.loc[data['make'].str.contains('朵唯', na=False), 'make'] = 'doov'
data.loc[data['make'].str.contains('sm', na=False), 'make'] = 'samsung'
data.loc[data['make'].str.contains('SAMSUNG', na=False), 'make'] = 'samsung'
data.loc[data['make'].str.contains('三星', na=False), 'make'] = 'samsung'
data.loc[data['make'].str.contains('GT-', na=False), 'make'] = 'samsung'
data.loc[data['make'].str.contains('格力', na=False), 'make'] = 'gree'
data.loc[data['make'].str.contains('Moto G', na=False), 'make'] = 'motorola'
data.loc[data['make'].str.contains('Moto', na=False), 'make'] = 'motorola'
data.loc[data['make'].str.contains('moto', na=False), 'make'] = 'motorola'
data.loc[data['make'].str.contains('摩托罗拉', na=False), 'make'] = 'motorola'
data.loc[data['make'].str.contains('诺基亚', na=False), 'make'] = 'nokia'
data.loc[data['make'].str.contains('Nokia', na=False), 'make'] = 'nokia'
data.loc[data['make'].str.contains('努比亚', na=False), 'make'] = 'nubia'
data.loc[data['make'].str.contains('美图', na=False), 'make'] = 'meitu'
data.loc[data['make'].str.contains('LG-', na=False), 'make'] = 'LG'
data.loc[data['make'].str.contains('联想', na=False), 'make'] = 'lenovo'
data.loc[data['make'].str.contains('rv:', na=False), 'make'] = 'RV'
data.loc[data['make'].str.contains('rv:', na=False), 'make'] = 'RV'
data.loc[data['make'].str.contains('小辣椒', na=False), 'make'] = 'xiaolajiao'
data.loc[data['make'].str.contains('HUAWEI', na=False), 'make'] = 'huawei'
data.loc[data['make'].str.contains('huawei', na=False), 'make'] = 'huawei'
data.loc[data['make'].str.contains('荣耀', na=False), 'make'] = 'huawei'
data.loc[data['make'].str.contains('华为', na=False), 'make'] = 'huawei'
data.loc[data['make'].str.contains('-L', na=False), 'make'] = 'huawei'
data.loc[data['make'].str.contains('al', na=False), 'make'] = 'huawei'
data.loc[data['make'].str.contains('Blade', na=False), 'make'] = 'zte'
data.loc[data['make'].str.contains('BLADE', na=False), 'make'] = 'zte'
data.loc[data['make'].str.contains('中兴', na=False), 'make'] = 'zte'
data.loc[data['make'].str.contains('海信', na=False), 'make'] = 'hisense'
data.loc[data['make'].str.contains('Linux', na=False), 'make'] = 'Linux'
data.loc[data['make'].str.contains('乐丰', na=False), 'make'] = 'lephone'
data.loc[data['make'].str.contains('百立丰', na=False), 'make'] = 'lephone'
data.loc[data['make'].str.contains('乐视', na=False), 'make'] = 'letv'
data.loc[data['make'].str.contains('XT', na=False), 'make'] = 'Sony'
data.loc[data['make'].str.contains('htc', na=False), 'make'] = 'htc'
data.loc[data['make'].str.contains('HTC', na=False), 'make'] = 'htc'
data.loc[data['make'].str.contains('ASUS', na=False), 'make'] = 'Asus'
data.loc[data['make'].str.contains('锤子', na=False), 'make'] = 'smartisan'
data.loc[data['make'].str.contains('oppo', na=False), 'make'] = 'oppo'
data.loc[data['make'].str.contains('pb', na=False), 'make'] = 'oppo'
data.loc[data['make'].str.contains('realme', na=False), 'make'] = 'oppo'
data.loc[data['make'].str.contains('天语', na=False), 'make'] = 'k touch'
data.loc[data['make'].str.contains('Tianyu', na=False), 'make'] = 'k touch'
data.loc[data['make'].str.contains('tianyu', na=False), 'make'] = 'k touch'
data.loc[data['make'].str.contains('酷派', na=False), 'make'] = 'coolpad'
data.loc[data['make'].str.contains('索尼', na=False), 'make'] = 'sony'
data.loc[data['make'].str.contains('SONY', na=False), 'make'] = 'sony'
data.loc[data['make'].str.contains('sony', na=False), 'make'] = 'sony'
data.loc[data['make'].str.contains('小米', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('mi', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m1', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m1s', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m2', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m2s', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m2a', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m3', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m6', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('xiaomi', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('redmi', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('魅族', na=False), 'make'] = 'meizu'
data.loc[data['make'].str.contains('360', na=False), 'make'] = '360'
data.loc[data['make'].str.contains('三星', na=False), 'make'] = 'samsung'
data.loc[data['make'].str.contains('赛博宇华', na=False), 'make'] = 'sop'
data.loc[data['make'].str.contains('金立', na=False), 'make'] = 'jinli'
data.loc[data['make'].str.contains('gionee', na=False), 'make'] = 'jinli'
data.loc[data['make'].str.contains('Gionee', na=False), 'make'] = 'jinli'
data.loc[data['make'].str.contains('vivo', na=False), 'make'] = 'vivo'
data.loc[data['make'].str.contains('VIVO', na=False), 'make'] = 'vivo'
data.loc[data['make'].str.contains('oneplus', na=False), 'make'] = 'oneplus'
data.loc[data['make'].str.contains('一加', na=False), 'make'] = 'oneplus'

In [47]:
full_data.make.unique()

array(['vivo', 'huawei', 'oppo', nan, 'EML-AL00', 'xiaomi', '360',
       'meizu', 'GIONEE', 'HONOR', 'samsung', 'Meizu', 'honor', 'jinli',
       'MI+5X', 'LeEco', 'ANE-AL00', 'htc', 'ZTE', 'Letv', 'BBK',
       'PCT-AL10', 'OnePlus', 'VCE-AL00', 'MIX%202', 'Meitu', 'V1816A',
       'PAR-AL00', 'PAR', 'LYA-AL00', 'GiONEE', 'MI8', 'CLT-TL01',
       'JMM-AL10', 'SMARTISAN', 'ARE-AL00', 'Huawei', '云狐', 'LeMobile',
       'nubia', 'COR-AL10', 'Sony', 'CLT-AL00', 'COL-AL10', 'DUB-AL00',
       'MI 9', 'Gree', 'LENOVO', 'coolpad', 'Generic', 'Hisense', 'meitu',
       'YuLong', '奇酷', 'SM-G9500', 'PBBM00', 'JSN-AL00', 'zte', 'Coolpad',
       'koobee', 'PBET00', 'nokia', 'PBAT00', 'AGM', 'oneplus', '/pado',
       'rockchip', 'philips', 'common', 'sast', 'blackshark', 'smartisan',
       '%2522OS105%2522', 'JSN-AL00a', '16th%20Plus',
       'GIONEE,F100A,F100A', 'INE-AL00', 'PBAM00', 'CMCC', 'KINGSUN',
       'PBCM30', 'gree', 'GREE', 'JKM-TL00', 'DOOV', 'YUFLY', 'Benign',
       'PBEM00', 

## 转换`nginxtime`属性为时间属性

In [58]:
data['time'] = pd.to_datetime(data.nginxtime[0] / 1000, unit='s') + timedelta(hours=8)

In [61]:
data['day'] = data.time.dt.day
data['hour'] = data.time.dt.hour

## 抽取`sid`中的时间戳

In [64]:
data.sid[1]

'b660d559-db97-4b5f-9bd2-2450cb89ce77-1560050740125'

In [65]:
data['sid_timestamp'] = data.sid.apply(lambda x:str(x).split('-')[-1]).astype(str)

In [76]:
data.sid_timestamp.head(3)

0    1560090701088
1    1560050740125
2    1560089145219
Name: sid_timestamp, dtype: object

- 判断请求时间与到达时间

In [70]:
data['nginxtime_sid_time_diff'] = data.nginxtime.astype('float') - data.sid_timestamp.astype('float')
data['nginxtime_sid_timestamp'] = (data.nginxtime==data.sid_timestamp.astype(str)).astype(int)

In [72]:
data.nginxtime_sid_time_diff.head(3)

0   -1.0
1   -2.0
2   -1.0
Name: nginxtime_sid_time_diff, dtype: float64

In [73]:
data.nginxtime_sid_timestamp.head(3)

0    0
1    0
2    0
Name: nginxtime_sid_timestamp, dtype: int32

## 转换`os`属性为大写

In [69]:
data.os = data.os.apply(lambda x:x.upper())

0

## 处理`model`特征

In [75]:
data.model.unique()

array(['V1809A', 'BLA-AL00', 'PBEM00', ..., 'SM-A7100', 'BTV-DL09',
       'LE X625'], dtype=object)

In [77]:
data['model'].replace('PACM00',"OPPO R15",inplace=True)
data['model'].replace('PBAM00',"OPPO A5",inplace=True)
data['model'].replace('PBEM00',"OPPO R17",inplace=True)
data['model'].replace('PADM00',"OPPO A3",inplace=True)
data['model'].replace('PBBM00',"OPPO A7",inplace=True)
data['model'].replace('PAAM00',"OPPO R15_1",inplace=True)
data['model'].replace('PACT00',"OPPO R15_2",inplace=True)
data['model'].replace('PABT00',"OPPO A5_1",inplace=True)
data['model'].replace('PBCM10',"OPPO R15x",inplace=True)

In [79]:
data.model.unique()

array(['V1809A', 'BLA-AL00', 'OPPO R17', ..., 'SM-A7100', 'BTV-DL09',
       'LE X625'], dtype=object)

In [81]:
data['big_model'] = data.model.apply(lambda x: str(x).split(' ')[0])

In [83]:
data['model_equal_make'] = (data.big_model == data.make).astype(int)

In [84]:
data['machine'] = data.make.astype(str) + '-' + data.model.astype(str)

In [89]:
len(data[data.h==0])

2404